# Imports

In [1]:
import numpy as np
import pandas as pd

# Carregando os dados

In [2]:
dtype={
       "region": object, 
       "city":object, 
       "parent_category_name":object,
       "category_name": object,
       "title":object,
       "description":object, 
       "price":np.float32, 
       "activation_date": object, 
       "user_type": object, 
       "image":object, 
       "image_top_1":np.float32, 
       "deal_probability":np.float32,
       "item_seq_number": np.int,
       "item_id": object,
       "param_1": object,
       "param_2": object,
       "param_3": object,
       "user_id": object
      }

df_train = pd.read_csv('./data/train_small.csv', dtype = dtype, encoding='utf8', memory_map =True)
df_test  = pd.read_csv('./data/test_small.csv' , dtype = dtype, encoding='utf8', memory_map =True)

text_columns = ['title','description','param_1','param_2','param_3']
categorical_columns = ["parent_category_name","category_name","user_type", "user_id"]

target_column = "deal_probability"

# Tratando dados faltantes
for column in text_columns:
    df_train[column].fillna(value='', inplace=True)
    df_test[column].fillna(value='', inplace=True)

# Merge dataFrames train e test
df_all_data = pd.concat([df_train,df_test])


# Exibindo os dados

In [3]:
df_train.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


# Eliminando colunas desnecessárias

In [4]:
columns_to_drop =['item_seq_number', 'item_id']
df_train = df_train.drop(columns=columns_to_drop)
df_test  = df_test.drop(columns=columns_to_drop)
df_train.head(5)

,region,city,parent_category_name,category_name,title,description,price,activation_date,user_type,image,image_top_1,deal_probability
0,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автокресло,Продам кресло от0-25кг,2200.0,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,Волгоградская область,Волгоград,Транспорт,Автомобили,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


# Processamento de texto corrido

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

#Stop words russas
stop_words_ru = open('data/stopwords.txt', encoding='utf8').read().split('\n')

def stemmed_words(doc):
    stemmer = RussianStemmer()
    analyzer = CountVectorizer().build_analyzer()
    return (stemmer.stem(w) for w in analyzer(doc))


#Pré processa os dados
for column in text_columns:
    
    # Criando tfIdfVectorizer
    tfidf = TfidfVectorizer(sublinear_tf=True, stop_words=stop_words_ru, analyzer=stemmed_words,
                        lowercase=True, min_df = 1, max_df=0.8)

    # fit 
    tfidf.fit(df_all_data[column])

    # transform
    df_train_column_transformed = pd.DataFrame(tfidf.transform(df_train[column]).toarray())
    df_test_column_transformed  = pd.DataFrame(tfidf.transform(df_test[column]).toarray())
     
    # apagando colunas antigas
    df_train = df_train.drop(columns=[column])
    df_test  = df_test.drop(columns=[column])
    
    # Mergeando dataFrames
    df_train = pd.concat([df_train, df_train_column_transformed], axis=1)
    df_test =  pd.concat([df_test, df_test_column_transformed], axis=1)
    


In [4]:
df_train.head(2)

,item_id,user_id,region,city,parent_category_name,category_name,price,item_seq_number,activation_date,user_type,...,26,27,28,29,30,31,32,33,34,35
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,400.0,2,2017-03-28,Private,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,3000.0,19,2017-03-26,Private,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Processamento de colunas categóricas

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

for column in categorical_columns:
    
     # fit labels
    label_enc = preprocessing.LabelEncoder() 
    label_enc.fit(df_all_data[column])
    
    # transform labels
    column_label_encoded       = label_enc.transform(df_all_data[column]).reshape(1, -1)    
    train_column_label_encoded = label_enc.transform(df_train[column]).reshape(1, -1)
    test_column_label_encoded  = label_enc.transform(df_test[column]).reshape(1, -1)
     
    # fit    
    one_hot_enc = OneHotEncoder()
    one_hot_enc.fit(column_label_encoded)
    
    # transform
    df_train_column_transformed = pd.DataFrame(one_hot_enc.transform(train_column_label_encoded).toarray())
    df_test_column_transformed  = pd.DataFrame(one_hot_enc.transform(test_column_label_encoded).toarray())
    
    # apagando colunas antigas
    df_train = df_train.drop(columns=[column])
    df_test  = df_test.drop(columns=[column])
    
    # Mergeando dataFrames
    df_train = pd.concat([df_train, df_train_column_transformed], axis=1)
    df_test =  pd.concat([df_test, df_test_column_transformed], axis=1)
    

ValueError: X has different shape than during fitting. Expected 81, got 49.

# Processando colunas geográficas   

### "Cruzando" as cidades e as regiões russas para obter coordenadas e adicionar ao DataFrame, onde possui os dados já processados.

In [33]:
from geopy import geocoders

df_city_region = df_train[["city", "region"]].apply(lambda l: " ".join(l), axis=1),

num_pontos = df_train.shape[0]
g = geocoders.Nominatim()

latitude  = np.ones(num_pontos)
longitude = np.ones(num_pontos)

for i in range(num_pontos):
    geocode = g.geocode(df_city_region[i], timeout=10, language='pt')
    latitude[i]  = geocode.latitude
    longitude[i] = geocode.longitude


GeocoderQueryError: HTTP Error 414: Request-URI Too Long

# Processamento de colunas temporais

In [34]:
//pendente

SyntaxError: invalid syntax (<ipython-input-34-4d8a6b34f68f>, line 1)